<a href="https://colab.research.google.com/github/Vikram-Ahuja/Stock_Portfolios/blob/main/ALPACA_Portfolio_HQM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import alpaca_trade_api as trade_api
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import datetime as dt
from scipy import stats
from statistics import mean
from math import floor 

In [ ]:
def return_calculator(stock,time):
    try:
        x = float(stocks_month_price[stock].Close[time])
        y = float(stocks_month_price[stock].Close[0])
    except:
        x = float(stocks_month_price[stock].Close[time+1])
        y = float(stocks_month_price[stock].Close[1])
    return_ = (y-x)*100/x
    percent = round(return_ , 4)
    return percent

def price_scrape(stock):
    url = f"https://in.finance.yahoo.com/quote/{stock}"
    data = rq.get(url).text
    soup = BeautifulSoup(data,"lxml")
    price1 = soup.find("span",class_ = "Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)")
    price = price1.text.replace(",","")

    return float(price)

In [ ]:
stocks = pd.read_csv("/content/drive/MyDrive/S&P_500.csv")
stocks_month_price ={}
rejected_stocks = []
for stock in stocks.Symbol: 
    try:
        d = pd.read_html(f"https://in.finance.yahoo.com/quote/{stock}/history?period1=1514764800&period2=1615334400&interval=1mo&filter=history&frequency=1mo&includeAdjustedClose=true")
        d = d[0][:25]
    
        del d['Volume']
        del d['Close*']
        d = d.rename(columns={"Adj. close**": "Close"})
        stocks_month_price[stock]  = d 

        for i in range(len(stocks_month_price[stock])):
            x = stocks_month_price[stock].Date[i]
            if "01-" not in x:
                stocks_month_price[stock].drop([i], inplace = True)

        stocks_month_price[stock].reset_index(drop=True,inplace =True)
    except:
        print(stock)
        rejected_stocks.append(stock)

CTXS


In [ ]:
for stock in rejected_stocks:
    try:
        d = pd.read_html(f"https://in.finance.yahoo.com/quote/{stock}/history?period1=1514764800&period2=1615334400&interval=1mo&filter=history&frequency=1mo&includeAdjustedClose=true")
        d = d[0][:25]
    
        del d['Volume']
        del d['Close*']
        d = d.rename(columns={"Adj. close**": "Close"})
        stocks_month_price[stock]  = d 

        for i in range(len(stocks_month_price[stock])):
            x = stocks_month_price[stock].Date[i]
            if "01-" not in x:
                stocks_month_price[stock].drop([i], inplace = True)

        stocks_month_price[stock].reset_index(drop=True,inplace =True)
    except:
        print(stock)
        rejected_stocks.append(stock)

In [ ]:
for stock in stocks.Symbol:
    if stocks_month_price[stock].empty == True:
        stocks_month_price.pop(stock)
        stock = stock.replace(".","-")
        d = pd.read_html(f"https://in.finance.yahoo.com/quote/{stock}/history?period1=1514764800&period2=1615334400&interval=1mo&filter=history&frequency=1mo&includeAdjustedClose=true")
        d = d[0][:25]
    
        del d['Volume']
        del d['Close*']
        d = d.rename(columns={"Adj. close**": "Close"})
        stocks_month_price[stock]  = d 

        for i in range(len(stocks_month_price[stock])):
            x = stocks_month_price[stock].Date[i]
            if "01-" not in x:
                stocks_month_price[stock].drop([i], inplace = True)

        stocks_month_price[stock].reset_index(drop=True,inplace =True)


In [ ]:
hqm_column = ["Symbol", "One_Year_return","One_Year_return_percentile" , "Six_month_return","Six_month_return_percentile","Three_month_return","Three_month_return_percentile","One_month_return","One_month_return_percentile","HQM_Score","Stocks_to_BUY"]
hqm_NASDAQ_50_Porfolio = pd.DataFrame(columns=hqm_column)

for stock in stocks.Symbol:
    try:
        One_Year_return = return_calculator(stock,12)
        Six_month_return = return_calculator(stock,6)
        Three_month_return = return_calculator(stock,3)
        One_month_return =return_calculator(stock,1)
        hqm_NASDAQ_50_Porfolio = hqm_NASDAQ_50_Porfolio.append(
            pd.Series(
                [
                stock,
                One_Year_return,
                "N/A", 
                Six_month_return,
                "N/A",
                Three_month_return,
                "N/A",
                One_month_return,
                "N/A",
                "N/A",
                "N/A"
                ],
                index =hqm_column
            ),
            ignore_index = True
            )
    except:
        print(stock)


In [ ]:
time_period = ["One_Year","Six_month","Three_month","One_month"]

for row in hqm_NASDAQ_50_Porfolio.index:
    for time in time_period:
        hqm_NASDAQ_50_Porfolio.at[row,f"{time}_return_percentile"] = float(stats.percentileofscore(hqm_NASDAQ_50_Porfolio[f"{time}_return"],hqm_NASDAQ_50_Porfolio.at[row,f"{time}_return"]))

In [ ]:
for row in hqm_NASDAQ_50_Porfolio.index:
    l = []
    for time in time_period:
        l.append(hqm_NASDAQ_50_Porfolio.at[row,f"{time}_return_percentile"])
    avg = mean(l)
    hqm_NASDAQ_50_Porfolio.at[row,"HQM_Score"] = avg

In [ ]:
NASDAQ_Portfolio_HQM = hqm_NASDAQ_50_Porfolio.sort_values("HQM_Score",ascending=False,ignore_index=True)[:50]

NASDAQ_Portfolio_HQM

In [ ]:
portfolio_size = float(input("Enter Portfolio Size: "))
position_size = portfolio_size/len(NASDAQ_Portfolio_HQM)

for i in range(len(NASDAQ_Portfolio_HQM)):
    NASDAQ_Portfolio_HQM.at[i , "Stocks_to_BUY"] = floor(position_size/price_scrape(NASDAQ_Portfolio_HQM.Symbol[i]))

Enter Portfolio Size: 100000


In [ ]:
NASDAQ_Portfolio_HQM

In [ ]:
api_key = "PKAMEQNMZCA2AHDJ3ODL"
api_secret_key="8K2sB0sjvJOpgL0lRi0FQFpM9SXRLsOqPlRkH6Qg"
api_base_url="https://paper-api.alpaca.markets"
api = trade_api.REST(key_id=api_key, secret_key=api_secret_key, base_url=api_base_url)

print (api.get_account().status)

for i in range (len(NASDAQ_Portfolio_HQM)):
    volume = NASDAQ_Portfolio_HQM.Stocks_to_BUY[i]
    sym = NASDAQ_Portfolio_HQM.Symbol[i]
    api.submit_order(sym,volume,"buy","market","gtc")

ACTIVE
